In [ ]:
#@title CNN-LSTM-PC
import tensorflow as tf
import numpy as np

# Seed value
seed_value= 0
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from keras.models import load_model
import pandas as pd
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from numpy import concatenate
from scipy.stats import boxcox
from math import exp
from math import log
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.metrics import mean_squared_error
from numpy import array
from pandas import concat
import warnings
warnings.filterwarnings("ignore")

# invert a boxcox transform for one value
def invert_boxcox(value, lam):
  # log case
  if lam == 0:
    return exp(value)
  # all other cases
  return exp(log(lam * value + 1) / lam)

# Convert series to supervised learning
def sup_struct_future(dataset, target, start,window,horizon):
  x_future = dataset[:,:-1]
  end = None
  X = []
  y = []
  X_f = []
  start = start + window
  if end is None:
    end = len(dataset) - horizon
  for i in range(start, end):
    indices = range(i-window, i)
    X.append(dataset[indices])
    indicey = range(i+1, i+1+horizon)  
    y.append(target[indicey])
    X_f.append(x_future[indicey])
  
  x_past = np.array(X)
  x_future_preds = np.array(X_f) 
  y_target = np.array(y)

  x_past_reshaped = x_past.reshape(x_past.shape[0],x_past.shape[1]*x_past.shape[2])
  x_future_preds_reshaped = x_future_preds.reshape(x_future_preds.shape[0],
                                                  x_future_preds.shape[1]*x_future_preds.shape[2])
  x_covs = np.concatenate([x_past_reshaped,x_future_preds_reshaped],axis=1)
  return x_covs, y_target

#Load county data
mydata = pd.read_csv('nas.csv', header=0)
dataIn = mydata[mydata.columns[1:8]] 
dateT = mydata[mydata.columns[0]]
dateT = DataFrame(dateT)
dateT['dateTime'] = dateT
dateT.drop('Unnamed: 0', axis = 1, inplace = True)
df_data = pd.concat((dateT, dataIn), axis=1)
df_data = df_data.drop(df_data.columns[0], axis = 1)

# Drop rows which contain any NaN values in all columns
df_data = df_data.dropna( how='all')

# Interpolate missing values
df_data = df_data.interpolate(method ='linear', limit_direction ='both', limit = 10000, axis=0)

# Split into train and test set
train_size = int(len(df_data) * 0.8)
train_data_1, test_data = df_data[0:train_size], df_data[train_size:]
val_size = int(len(train_data_1) * 0.1)
val_data = train_data_1[-val_size:]
train_data = train_data_1[:train_size-val_size]

# move target variable to the last column to align with surpervise learning struture
new_train =  pd.DataFrame(train_data,columns=['Temp','Preci','Gust','Wind','Windir','LAI','Total Outages'])
new_test  =  pd.DataFrame(test_data,columns=['Temp','Preci','Gust','Wind','Windir','LAI','Total Outages'])
new_val   =  pd.DataFrame(val_data,columns=['Temp','Preci','Gust','Wind','Windir','LAI','Total Outages'])
raw_train =  pd.DataFrame(train_data,columns=['Temp','Preci','Gust','Wind','Windir','LAI','Total Outages'])
raw_test  =  pd.DataFrame(test_data,columns=['Temp','Preci','Gust','Wind','Windir','LAI','Total Outages'])

# power transform target variable to be gausian like :TRAIN
box_power = new_train[['Total Outages']]
box_power = np.array(box_power)
box_power = box_power.reshape(box_power.shape[0]*1) 
transformed, lmbda = boxcox(box_power)
transformed = transformed.reshape(-1,1) 

# Apply power transform to target validation data  :VAL
box_power_val = new_val[['Total Outages']]
box_power_val = np.array(box_power_val)
box_power_val = box_power_val.reshape(box_power_val.shape[0]*1)  
transformed_val = boxcox(box_power_val,lmbda)
transformed_val = transformed_val.reshape(-1,1)  

# Apply power transform to target validation data  :TEST
box_power_test = new_test[['Total Outages']]
box_power_test = np.array(box_power_test)
box_power_test = box_power_test.reshape(box_power_test.shape[0]*1)  
transformed_test = boxcox(box_power_test,lmbda)
transformed_test = transformed_test.reshape(-1,1)  

# Smothing targets
dwAvg = 6 # 6hrs rolling avearge
y_train_mean = pd.DataFrame(transformed)
y_train_mean = y_train_mean.rolling(dwAvg).mean()
y_train_mean = y_train_mean.interpolate(method ='linear', limit_direction ='both', limit = 100, axis=0)
y_train_mean = np.array(y_train_mean)
y_train_mean = y_train_mean.reshape(-1,1)
new_train[['Total Outages']] = y_train_mean

# Val fold            
y_val_mean = pd.DataFrame(transformed_val)
y_val_mean = y_val_mean.rolling(dwAvg).mean()
y_val_mean = y_val_mean.interpolate(method ='linear', limit_direction ='both', limit = 100, axis=0)
y_val_mean = np.array(y_val_mean)
y_val_mean = y_val_mean.reshape(-1,1)
new_val[['Total Outages']] = y_val_mean

# Test fold            
y_test_mean = pd.DataFrame(transformed_test)
y_test_mean = y_test_mean.rolling(dwAvg).mean()
y_test_mean = y_test_mean.interpolate(method ='linear', limit_direction ='both', limit = 100, axis=0)
y_test_mean = np.array(y_test_mean)
y_test_mean = y_test_mean.reshape(-1,1)
new_test[['Total Outages']] = y_test_mean

# Scaling
# Train
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled_data = X_scaler.fit_transform(new_train[['Temp', 'Preci', 'Gust', 'Wind', 'Windir', 'LAI',
                                      'Total Outages']])
Y_scaled_data = Y_scaler.fit_transform(new_train[['Total Outages']])

# Validation
X_val_scaled_data = X_scaler.transform(new_val[['Temp', 'Preci', 'Gust', 'Wind', 'Windir', 'LAI',
                                      'Total Outages']])
Y_val_scaled_data = Y_scaler.transform(new_val[['Total Outages']]) 

# Test
X_test_scaled_data = X_scaler.transform(new_test[['Temp', 'Preci', 'Gust', 'Wind', 'Windir', 'LAI',
                                      'Total Outages']])
Y_test_scaled_data = Y_scaler.transform(new_test[['Total Outages']]) 

lag = 6 # look back
h = 6 # forecast horizon
epoch = 100

# Call the supervised learning method
x_train, y_train = sup_struct_future(X_scaled_data,Y_scaled_data,0,lag,h)
x_vali, y_vali = sup_struct_future(X_val_scaled_data,Y_val_scaled_data,0,lag,h)

# Reshape arrays for CNN-LSTM
x_train_reshaped = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_vali_reshaped = x_vali.reshape(x_vali.shape[0],x_vali.shape[1],1)
y_train_reshaped = y_train.reshape(y_train.shape[0],y_train.shape[1]*1)
y_vali_reshaped = y_vali.reshape(y_vali.shape[0],y_vali.shape[1]*1)

# Build CNN and LSTM networks
from tensorflow.keras.callbacks import LearningRateScheduler, Callback, TensorBoard, ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense, Lambda, Flatten, Activation, Concatenate
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Conv1D,CuDNNLSTM
from keras.layers import MaxPooling1D
from keras.layers import Dropout
from matplotlib import pyplot
from matplotlib import pyplot

tf.keras.backend.clear_session() # clears keras session
checkpoint_filepath = 'source.h5'
inputs1=Input(shape=(x_train_reshaped.shape[-2:]))

# CNN Network
x = Conv1D(filters=32,kernel_size=2,padding='same',activation="relu")(inputs1)
x = Conv1D(filters=32,kernel_size=2,padding='same',activation="relu")(x)
x = Conv1D(filters=32,kernel_size=2,padding='same',activation="relu")(x)
x = Conv1D(filters=32,kernel_size=2,padding='same',activation="relu")(x)
x = MaxPooling1D(pool_size=2,padding='same')(x)
x = Dropout(0.1)(x)
x = Flatten()(x)

# LSTM network
ls = CuDNNLSTM(50)(inputs1)
ls = Activation('relu')(ls)  
ls = Dropout(0.1)(ls)

# Concatenate outputs from CNN and LSTM networks  
merged = Concatenate()([x, ls])   
# Pass outputs to dense and lamda layers for optimization
merged = Dense(10,activation='relu')(merged) 
merged = Dense(h,activation='linear')(merged)
merged = Lambda(lambda x: x * 10)(merged)  
model=Model(inputs1,merged)

# Optimizer and loss function 
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-6, momentum = 0.9)
model.compile(loss=tf.keras.losses.Huber(),optimizer=optimizer)

# Learning rate schedules  
es = [EarlyStopping(monitor="val_loss", patience = 100, mode = 'min', verbose =1)]   
mc = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', mode='min', 
                      verbose=0, save_best_only=True) 
reduce_lr_n = ReduceLROnPlateau(monitor='loss', factor=0.8,
                            patience=5, min_lr=0.0000001, verbose=0,mode='min')
callbacks=[es,mc,reduce_lr_n]
# Fit 
history = model.fit(x_train_reshaped,y_train_reshaped,validation_data=(x_vali_reshaped,
                    y_vali_reshaped),epochs=epoch,verbose=1,callbacks=callbacks)
# Save model architecture
dot_img_file = 'source.png'
tf.keras.utils.plot_model(model,to_file=dot_img_file, show_shapes=True)

# Visualize learning curves   
pyplot.title('Learning Curves')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.legend()
pyplot.show()
# Save best model
model.save(checkpoint_filepath)
# load the saved model
model = tf.keras.models.load_model(checkpoint_filepath)

# Prepare test data
predictions = []
all_preds = []
x_future = X_test_scaled_data[:,:-1]
end = None
start = 0
X = []
y = []
X_f = []
start = start + lag
if end is None:
  end = len(X_test_scaled_data) - h
for i in range(start, end, lag):
  indices = range(i-lag, i)
  X.append(X_test_scaled_data[indices])  
  indicey = range(i+1, i+1+h)
  y_future = Y_test_scaled_data[indicey] 
  y_future = y_future.reshape(y_future.shape[1],y_future.shape[0]) 
  y.append(y_future)
  X_f.append(x_future[indicey])
  x_past = np.array(X)
  x_future_preds = np.array(X_f)
 
 
  x_past_reshaped = x_past.reshape(x_past.shape[0],x_past.shape[1]*x_past.shape[2])
  x_future_preds_reshaped = x_future_preds.reshape(x_future_preds.shape[0],
                                                x_future_preds.shape[1]*x_future_preds.shape[2])
  x_covs = np.concatenate([x_past_reshaped,x_future_preds_reshaped],axis=1)
  x_covs_reshaped = x_covs.reshape(x_covs.shape[0],x_covs.shape[1],1)
  y_pred = model.predict(x_covs_reshaped)
  predictions.append(y_pred) 
  X = []  
  X_f = []

# Compute test scores
y_target = np.array(y)
y_p = np.array(predictions)
y_last_h_reshaped = y_p.reshape(y_p.shape[0],y_p.shape[1]*y_p.shape[2])
y_last_h =y_last_h_reshaped[:,-1]
yhat = y_last_h.reshape(-1,1)
pred_Inverse = Y_scaler.inverse_transform(yhat)
# Inverse power transform of forecast values
y_b_inv = [invert_boxcox(x, lmbda) for x in pred_Inverse]
y_b_inv = np.array(y_b_inv)
# Inverse power transform of actual values
y_test_reshaped = y_target.reshape(y_target.shape[0],y_target.shape[1]*y_target.shape[2])
y_test_actual = y_test_reshaped[:,-1]
y_test_actual = y_test_actual.reshape(-1,1)
actual_Inverse = Y_scaler.inverse_transform(y_test_actual) 
y_actual_inv = [invert_boxcox(x, lmbda) for x in actual_Inverse]
y_actual_inv = np.array(y_actual_inv)

y_actual = y_actual_inv
y_train_actual = raw_train['Total Outages']
y_train_actual = y_train_actual.values

pred = y_b_inv
actual = y_actual[-len(pred):]

mse = mean_squared_error(actual, pred)
# calculate rmse
rmse = sqrt(mse)  
# print rmse
print('Average Test RMSE: %.3f' % (rmse))
from sklearn import metrics
print(f'R2 is : {metrics.r2_score(actual, pred)}',end='\n\n')

# Visualizations
import matplotlib.pyplot as plt
fig, axes = plt.subplots(2, 1, figsize=(12, 8))
fig.tight_layout(pad=6.0)
axes[0].set_title("Actual & forecasted test samples")
axes[0].set_xlabel("Time Step")
axes[0].set_ylabel("Customer Outages")
real = DataFrame(actual)
real = real.values

x_axis = np.arange(y_train_actual.shape[0] + pred.shape[0])
axes[0].plot(x_axis[:y_train_actual.shape[0]], y_train_actual, alpha=0.75,label='Train Data')
axes[0].legend(loc="upper right")
axes[0].scatter(x_axis[y_train_actual.shape[0]:], pred, alpha=0.4,label='Forecast')
axes[0].legend(loc="upper right")
axes[0].scatter(x_axis[y_train_actual.shape[0]:], y_actual[-len(pred):], alpha=0.4, marker='x',label='Test Data')
axes[0].legend(loc="upper right")

new_x_axis = np.arange(300)
axes[1].plot(new_x_axis[:real.shape[0]], real[:300], alpha=0.75,label='Actual')
axes[1].legend(loc="upper right")
axes[1].plot(new_x_axis[:real.shape[0]],pred[:300],alpha=0.4,label='Predicted',color='r')
axes[1].legend(loc="upper right")
axes[1].set_title("Actual & forecasted test samples:First 100 time steps in expanded view")
axes[1].set_xlabel("Time Step")
axes[1].set_ylabel("Customer Outages")
plt.show()